# Зависимости

In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np

from mycsv import split_file
from dataset import load_dataset
from transform import Converter
from dataprocess import lzip

# Загрузка модели

In [ ]:
MODEL_PATH = 'fnp_bilstm_maxlen250_e20_p770.h5'
model = load_model(MODEL_PATH)
model.summary()

# Загрузка данных для исследования

In [ ]:
split_file('dataset.csv', split_ratio=0.8)

После разделения датасета в дериктории появиться два файла (тренировочный сет и тестовый).

* `maxlen` задает максимальную длину последовательности узлов функции (средняя длина последовательности ~50)

* `verbose` задает многословность функций обработчиков (то столько будет выводится информации во время обратоки); доступные значения от 0 до 3

In [ ]:
train_file_name = 'train_dataset.csv'
test_file_name = 'test_dataset.csv'
maxlen = 250
verbose = 1

Загрузка конвертеров из файлов.

In [ ]:
data_converter = Converter.from_file('dictionary_1697.json')
marks_converter = Converter.from_file('dictionary_97.json')

Загрузка данных для исследования.

In [ ]:
dataset = load_dataset(train_file_name, 
                       test_file_name, 
                       maxlen,
                       data_converter, 
                       marks_converter,
                       verbose
                       )
x_train, y_train, x_test, y_test, data_converter, marks_converter = dataset

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Доля верных ответов на тестовых данных составляет:',round(scores[1]* 100, 4),'%')

Соединение датасетов и добавление строковых лейбелов

In [ ]:
# Создание списка лейбелов в соотвествии с OHE
labels = []

for y in y_train:
  labels.append(marks_converter.index_to_item(np.where(y == 1)[0][0]))
train = list(lzip(x_train, y_train, labels))
labels.clear()

for y in y_test:
  labels.append(marks_converter.index_to_item(np.where(y == 1)[0][0]))
test = list(lzip(x_test, y_test, labels))

# Исследование работы модели на исходном датасете

На данном этапе следует проанализировать работу модели на примерах тестовой выборки, с целью исключения функций, которые приводят к коллизиям.

Для этого можно проанализировать следующие случаи:

1. Вероятность правильного определения функции для каждого имени. И поиск самых плохоугадываемых функций.

2. Поиск функций с отличным названием, но схожим назначением.

Функция для предсказания одного примера

In [ ]:
def make_prediction(model, sample):
  array = np.array(sample)
  array = (np.expand_dims (array, 0))
  return model.predict(array)[0]

**Добавление к данным значений предсказания модели.**

Предсказание представляет из себя массив вероятностей. Каждый элемент массива стоит в соответствии с элементом one hot encoding (OHE) метки.

Пример:

OHE: `[0, 0, 1, 0...] # единица - метка` 

predict: `[0.001, 0.003, 0.8, 0.00001...] # предсказывает метку с вероятностью 80 %`

Также к данным добавляется `index` метки в OHE. В примере выше `index = 2`.

Таким образом получается следующая структура данных для одного экземпляра данных:

`[data_tensor, OHE, label, prediction, index]`

* `data_tensor: numpy.ndarray` - вектор по которому модель делает предсказание (последовательность токенов в цифровом формате)
* `OHE: numpy.ndarray` - метка в формате One Hot Encoding
* `label: str` - название функции (метка в формате str)
* `prediction: numpy.ndarray` - вектор с вероятностями (отображения на OHE)
* `index: int` - индекс метки в OHE.

In [ ]:
# TODO: data -> dict
for sample in test:
  predict = make_prediction(model, sample[0])
  index = int(np.where(sample[1] == 1)[0][0])
  sample.append(predict)
  sample.append(index)

### Вероятность правильного определения функции для каждого имени.

Общее количсество функций для каждого имени.

In [ ]:
def count_func_names(data):
  counter_names = {}
  for i, sample in enumerate(data):
    name = sample[2]
    counter_names[name] = counter_names.get(name, 0) + 1
  return counter_names

Сумма вероятностей для каждого имени функции (оптимизировать).

In [ ]:
def count_func_probes(data):
  counter_prob = {}
  for sample in data:
    predict = sample[3]
    index = sample[4]
    name = sample[2]
    counter_prob[name] = counter_prob.get(name, 0) + predict[index]
  return counter_prob

Сумма всех вероятностей для одного имени деленное на количество функций в выборке дает среднюю вероятность правильного определения функции.
$$
counter\_prob_i / counter\_names_i = result_i
$$

In [ ]:
counter_prob = count_func_probes(test)
counter_names = count_func_names(test)

result = {}
for name, probs_sum in counter_prob.items():
  result[name] = probs_sum / counter_names[name]

In [ ]:
result = [(key, value) for key, value in result.items()]
result.sort(key=lambda x: x[1])

Чем меньше значение названия функции, тем хуже она определяется моделью.

In [ ]:
result[:50]

# Поиск похожих функций

Разделение примеров на две категории:

* `misses` - ошибочные предсказания модели
* `hits` - правильные предсказания модели

Структура: `[actual_name, precited_name, correct_bool]`

In [ ]:
matching = []
for sample in test:
  name = sample[2]
  predict = sample[3]
  prediced_index = np.argmax(predict)
  predicted_name =  marks_converter.index_to_item(prediced_index)
  miss = name != predicted_name

  index = sample[4]
  name_prob = predict[index]
  predicted_name_prob = predict[prediced_index]
  matching.append((name, name_prob, predicted_name, predicted_name_prob, miss))

In [ ]:
total = len(matching)
misses = list(filter(lambda x: x[4] == True, matching))
hits = list(filter(lambda x: x[4] == False, matching))
assert total == len(misses) + len(hits)

print(len(misses), len(hits))

In [ ]:
misses.sort(key=lambda x: x[1])
misses[-100:]

In [ ]:
hits.sort(key=lambda x: x[1])
hits[:100]

In [ ]:
# cell для хранения
for key in marks_converter.item_index_dict:
  if 'add' in key: print(key) 
list(filter(lambda x: x[0] == 'close', hits))
sorted(list(filter(lambda x: x[0] == 'close', misses)), key=lambda x: x[2])

In [ ]:
deltas = []
for name in marks_converter.item_index_dict:
  misses_names = []
  for miss in misses:
    misses_names.append(miss[0])
  miss_rate = misses_names.count(name)/len(misses)

  hits_names = []
  for hit in hits:
    hits_names.append(hit[0])
  hit_rate = hits_names.count(name)/len(hits)

  delta = hit_rate - miss_rate
  deltas.append((name, delta))

In [ ]:
deltas.sort(key=lambda x: x[1])
deltas

# Исследование работы модели на реальных примерах

**Загрузка реальных сниппетов функций**

In [ ]:
SNIPPETS_PATH = 'real_snippets.csv'
snippets = load_dataset(SNIPPETS_PATH, 
                       SNIPPETS_PATH, 
                       maxlen,
                       data_converter, 
                       marks_converter,
                       verbose
                       )
x_snippets, y_snippets, x_test, y_test, data_converter, marks_converter = snippets